<a href="https://colab.research.google.com/github/brandonnassir/Predictive_Policing_Model/blob/main/predictive_policing_model_3_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import the neccessary libraries
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
import matplotlib.animation as animation
from IPython.display import Image
import geopandas as gpd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.metrics import accuracy_score, classification_report, precision_score
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive/')
#Change the current working directory to the path of Google Cloud Drive
path="/content/drive/My Drive/Colab Notebooks/"
os.chdir(path)
os.listdir(path)
#Use the wget command to download the dataset to this path
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/data.csv
file_path = '/content/drive/MyDrive/Colab Notebooks/combined_data.csv'

Mounted at /content/drive/
--2024-06-17 13:50:30--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/data.csv
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 108.157.254.124, 108.157.254.15, 108.157.254.121, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|108.157.254.124|:443... connected.
HTTP request sent, awaiting response... 403 Forbidden
2024-06-17 13:50:30 ERROR 403: Forbidden.



In [ ]:
df = pd.read_csv(file_path)
df.head(20)

,DATE OCC,Hour of Day,Months,Days of Week,0,1,2,3,4,5,...,50,51,52,53,54,55,56,57,58,59
0,2020-01-01,0.0,1.0,4.0,0.000000,1.896940,1.716223,0.000000,0.000000,0.000000,...,0.000000,0.000000,1.393447,0.000000,0.362277,0.000000,0.160257,0.000000,0.333328,0.000000
1,2020-01-01,1.0,1.0,4.0,0.000000,0.686274,0.041985,0.000000,0.000000,0.532548,...,0.709240,0.198785,0.329118,0.000000,0.840384,0.000000,0.000000,0.000000,0.327572,0.281545
2,2020-01-01,2.0,1.0,4.0,0.000000,0.760256,0.000000,0.000000,0.151034,0.000000,...,0.000000,0.089470,0.451233,0.129547,0.096640,0.000000,0.000000,0.000000,0.317395,0.000000
3,2020-01-01,3.0,1.0,4.0,0.000000,0.837869,0.497633,0.000000,0.000000,0.136928,...,0.000000,0.099430,0.349515,0.304322,0.387116,0.000000,0.223926,0.041582,0.044034,0.028660
4,2020-01-01,4.0,1.0,4.0,0.000000,0.366070,0.395046,0.000000,0.000000,0.067603,...,0.000000,0.373876,0.149449,0.000000,0.792519,0.000000,0.182992,0.000000,0.000000,0.152160
5,2020-01-01,5.0,1.0,4.0,0.045271,0.736405,0.000000,0.000000,0.114290,0.230526,...,0.176646,0.170593,0.192686,0.017320,0.035072,0.000000,0.158550,0.000000,0.198398,0.000000
6,2020-01-01,6.0,1.0,4.0,0.000000,0.122619,0.188942,0.000000,0.000000,0.161990,...,0.000000,0.272392,0.240340,0.000000,0.560388,0.000000,0.176199,0.000000,0.073637,0.000000
7,2020-01-01,7.0,1.0,4.0,0.000000,0.620213,0.408612,0.058511,0.107069,0.075612,...,0.036446,0.000000,0.464607,0.274266,0.404927,0.000000,0.129521,0.004807,0.071081,0.000000
8,2020-01-01,8.0,1.0,4.0,0.000000,1.430662,0.573339,0.000000,0.027917,0.387142,...,0.579125,0.000000,0.662120,0.361142,0.274215,0.000000,0.229177,0.000000,0.000000,0.000000
9,2020-01-01,9.0,1.0,4.0,0.000000,1.168270,0.631791,0.047208,0.000000,0.281198,...,0.000000,0.140309,0.548994,0.000000,0.338665,0.000000,0.160728,0.224059,0.000000,0.140151


In [ ]:
df.shape

(35040, 64)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# Load your data
data = df

# 1. Encode Date Features

# Convert DATE OCC to datetime
data['DATE OCC'] = pd.to_datetime(data['DATE OCC'])

# Extract day of the year
data['DayOfYear'] = data['DATE OCC'].dt.dayofyear

# Normalize day of the year
data['DayOfYear'] = data['DayOfYear'] / 365.0

# Sine and cosine transformations for cyclical features
data['HourOfDay_sin'] = np.sin(2 * np.pi * data['Hour of Day'] / 24)
data['HourOfDay_cos'] = np.cos(2 * np.pi * data['Hour of Day'] / 24)
data['Month_sin'] = np.sin(2 * np.pi * data['Months'] / 12)
data['Month_cos'] = np.cos(2 * np.pi * data['Months'] / 12)
data['DayOfWeek_sin'] = np.sin(2 * np.pi * data['Days of Week'] / 7)
data['DayOfWeek_cos'] = np.cos(2 * np.pi * data['Days of Week'] / 7)

# Drop the original columns
data.drop(columns=['DATE OCC', 'Hour of Day', 'Months', 'Days of Week'], inplace=True)

# 2. Scale the Data

scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data)

# 3. Prepare Sequences for LSTM

def create_sequences(data, time_steps=10):
    sequences = []
    for i in range(len(data) - time_steps):
        seq = data[i:i + time_steps]
        sequences.append(seq)
    return np.array(sequences)

time_steps = 24 # You can change this based on your problem
sequences = create_sequences(scaled_data, time_steps)

# Now, split the sequences into input (X) and output (y)
X = sequences[:, :-1, :]
y = sequences[:, -1, :60] # Assuming columns 0 - 59 are the targets

# 4. Split the Data into Training and Test Sets

# Define the point where you want to split the data (e.g., 80% for training)
split_point = int(len(X) * 0.8)

# Split the data
X_train, X_test = X[:split_point], X[split_point:]
y_train, y_test = y[:split_point], y[split_point:]


# Check the shapes of the resulting datasets
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)


X_train shape: (28012, 23, 67)
X_test shape: (7004, 23, 67)
y_train shape: (28012, 60)
y_test shape: (7004, 60)


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Define the number of features
n_features = X_train.shape[2]
sequence_length = X_train.shape[1]

learning_rate = 0.00001
batch_size = 64  # Change this as neede

# Build the model
model = Sequential([
    # LSTM layer with 50 units
    LSTM(50, return_sequences=False, input_shape=(sequence_length, n_features)),

    # Dense layer with 64 units and ReLU activation
    Dense(64, activation='relu'),

    # Dense layer with 128 units and ReLU activation
    Dense(128, activation='linear'),

    # Dense layer with 256 units and linear activation
    Dense(256, activation='sigmoid'),

    # Output layer with the number of units equal to the target variables and sigmoid activation
    Dense(units=y_train.shape[1], activation='relu')
])

# Compile the model
optimizer = Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss='mean_squared_error')

# Print the model summary
model.summary()




Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 50)                23600     
                                                                 
 dense (Dense)               (None, 64)                3264      
                                                                 
 dense_1 (Dense)             (None, 128)               8320      
                                                                 
 dense_2 (Dense)             (None, 256)               33024     
                                                                 
 dense_3 (Dense)             (None, 60)                15420     
                                                                 
Total params: 83628 (326.67 KB)
Trainable params: 83628 (326.67 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
learning_rate = 0.00001
optimizer = Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss='mean_squared_error')

In [ ]:
# Train the model
history = model.fit(X_train, y_train, epochs=4800, batch_size=1028)

# Evaluate the model on the test set
test_loss = model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss}')

Epoch 1/4800
28/28 [==============================] - 4s 6ms/step - loss: 0.0575
Epoch 2/4800
28/28 [==============================] - 0s 6ms/step - loss: 0.0498
Epoch 3/4800
28/28 [==============================] - 0s 6ms/step - loss: 0.0434
Epoch 4/4800
28/28 [==============================] - 0s 6ms/step - loss: 0.0381
Epoch 5/4800
28/28 [==============================] - 0s 6ms/step - loss: 0.0335
Epoch 6/4800
28/28 [==============================] - 0s 6ms/step - loss: 0.0296
Epoch 7/4800
28/28 [==============================] - 0s 6ms/step - loss: 0.0262
Epoch 8/4800
28/28 [==============================] - 0s 6ms/step - loss: 0.0234
Epoch 9/4800
28/28 [==============================] - 0s 6ms/step - loss: 0.0209
Epoch 10/4800
28/28 [==============================] - 0s 6ms/step - loss: 0.0188
Epoch 11/4800
28/28 [==============================] - 0s 6ms/step - loss: 0.0171
Epoch 12/4800
28/28 [==============================] - 0s 6ms/step - loss: 0.0157
Epoch 13/4800
28/28 [====

KeyboardInterrupt: 

In [ ]:
#model.save('model_3.keras')

In [ ]:
from keras.models import load_model

# Path to your model
#model_path_1 = '/content/drive/My Drive/Colab Notebooks/model_3_1.keras'
model_path_2 = '/content/drive/My Drive/Colab Notebooks/autoencoder_model.keras'

# Load the saved model
#model_3_1 = load_model(model_path_1)
model_3_1 = model
autoencoder_model = load_model(model_path_2)

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense

# Define the encoder model structure
#input_dim = data_x.shape[1]
input_dim = 282
encoding_dim = autoencoder_model.layers[2].output_shape[1]  # Assuming the encoding layer is the second Dense layer

# Input for the decoder
encoded_input = Input(shape=(encoding_dim,))

# Recreate the decoder layers
decoder_layer1 = autoencoder_model.layers[-3]
decoder_layer2 = autoencoder_model.layers[-2]
decoder_output_layer = autoencoder_model.layers[-1]

# Build the decoder model
decoder = Model(encoded_input, decoder_output_layer(decoder_layer2(decoder_layer1(encoded_input))))


In [ ]:
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data.iloc[:, :60])  # Scale only the first 60 columns

In [ ]:
def inverse_transform(scaled_data, scaler):
    return scaler.inverse_transform(scaled_data)

# Example: transform LSTM predictions (embeddings) with the decoder
embeddings = model_3_1.predict(X_test)  # Assume this gives you the embeddings


# Inverse transform the embeddings
embeddings_inverse = inverse_transform(embeddings, scaler)


219/219 [==============================] - 1s 2ms/step


In [ ]:
# Use the decoder to transform embeddings back to the original space
decoded_predictions = decoder.predict(embeddings_inverse)

# Now, you can evaluate the performance or use the decoded predictions as needed


219/219 [==============================] - 0s 1ms/step


In [ ]:
decoded_y_test = decoder.predict(y_test)

219/219 [==============================] - 0s 1ms/step


In [ ]:
decoded_predictions = (decoded_predictions > 0.5).astype(int)
decoded_y_test = (decoded_y_test > 0.5).astype(int)

In [ ]:
# Assuming 'data' is your original training data and 'autoencoder' is your train

# Count correct predictions
correct_predictions = np.sum(decoded_predictions == decoded_y_test)

print(f"Number of correct element-wise predictions: {correct_predictions}")
print(f"Number of total elements: {int(decoded_predictions.shape[0]*int(decoded_predictions.shape[1]))}")

Number of correct element-wise predictions: 1908941
Number of total elements: 1975128


In [ ]:
right = 0
wrong = 0
one_count_y = 0
one_count_p = 0
true_positive = 0
false_positive = 0
false_negative = 0
for j in range(len(decoded_predictions)):
    for i in range(len(decoded_predictions[j])):
        if decoded_y_test[j][i] == decoded_predictions[j][i]:
            right = right + 1
        else:
            wrong = wrong + 1

        if decoded_y_test[j][i] == 1:
            one_count_y = one_count_y + 1

        if decoded_predictions[j][i] == 1:
            one_count_p = one_count_p + 1

        if (decoded_predictions[j][i] == decoded_y_test[j][i]) and (decoded_y_test[j][i] == 1):
            true_positive = true_positive + 1

        if (decoded_predictions[j][i] == 1) and (decoded_y_test[j][i] == 0):
            false_positive = false_positive + 1

        if (decoded_predictions[j][i] == 0) and (decoded_y_test[j][i] == 1):
            false_negative = false_negative + 1

precision = true_positive/(true_positive+false_positive)
recall = true_positive/(true_positive + false_negative)
f1_score = 2*(precision*recall)/(precision+recall)

In [ ]:
print("\nAccuracy:", round(right/(right+wrong),2))
print("\nPrecisions", round(true_positive/(true_positive+false_positive),2))
print("\nRecall:", round(true_positive/(true_positive + false_negative),2))
print("\nF1 Score,", round(f1_score,2))


Accuracy: 0.97

Precisions 0.92

Recall: 0.95

F1 Score, 0.94


In [ ]:
print("Total number of times model predicted crime event outcome correctly:",right)
print("\nTotal number of times model predicted crime event outcome incorrectly:",wrong)
print("\nTotal number of times model predicted crime would occur in Test Set:",one_count_p)
print("\nTotal number of times crime actually occured in Test Set:",one_count_y)
print("\nNumber of times model predicted crime occuring and crime did occur:",true_positive)
print("\nNumber of times model predicted crime occuring and crime did NOT occur:", false_positive)

Total number of times model predicted crime event outcome correctly: 1908941

Total number of times model predicted crime event outcome incorrectly: 66187

Total number of times model predicted crime would occur in Test Set: 522069

Total number of times crime actually occured in Test Set: 509038

Number of times model predicted crime occuring and crime did occur: 482460

Number of times model predicted crime occuring and crime did NOT occur: 39609
